In [1]:
import os
os.environ["OMP_PROC_BIND"] = os.environ.get("OMP_PROC_BIND", "true")

import numpy as np
import matplotlib.pyplot as plt
from pix_transform_diff_models import PixTransform_diff_models
from pix_transform_net_diff_models import PixTransformNetBase, PixTransformNetDeeper, PixTransformNetAttention, PixTransformNetMultiScale, PixTransformNetResidual
from baselines.baselines import bicubic
from utils.utils import downsample,align_images
from utils.plots import plot_result
import cv2
import torch
from scipy import ndimage
import json
from skimage.color import label2rgb
from PIL import Image
from scipy.stats import zscore
import tifffile
from skimage import exposure
import pandas as pd

In [ ]:
# Function to adjust image contrast
def enhance_contrast(image):
    # Perform contrast stretching
    p2, p98 = np.percentile(image, (2, 98))
    image_rescale = exposure.rescale_intensity(image, in_range=(p2, p98))
    return image_rescale

tissue = 'C_01'
imc_dir = f'Y:\\coskun-lab\\Efe\\gsr_imc\\datasets\\integration_mass_cytometry_data\\Coregistration\\Coregistration_files\\IMC_images+masks\\Tissue_{tissue}\\channels\\'
for imc_img_dir in os.listdir(imc_dir)[:-1]:
    if imc_img_dir == 'Thumbs.db':
        continue
    imc_img = tifffile.imread(imc_dir + imc_img_dir)
    imc_img = enhance_contrast(imc_img)
    plt.imshow(imc_img)
    plt.title(imc_img_dir)
    plt.show()

In [ ]:
import tifffile
import matplotlib.pyplot as plt

imc_dna = f'Y:\\coskun-lab\\Efe\\gsr_imc\\datasets\\integration_mass_cytometry_data\\Coregistration\\Coregistration_files\\IMC_images+masks\\Tissue_{tissue}\\channels\\191Ir_DNA1.ome.tiff'
imc_vimentin = f'Y:\\coskun-lab\\Efe\\gsr_imc\\datasets\\integration_mass_cytometry_data\\Coregistration\\Coregistration_files\\IMC_images+masks\\Tissue_{tissue}\\channels\\194Pt_Vimentin.ome.tiff'
imc_bcat = f'Y:\\coskun-lab\\Efe\\gsr_imc\\datasets\\integration_mass_cytometry_data\\Coregistration\\Coregistration_files\\IMC_images+masks\\Tissue_{tissue}\\channels\\196Pt_Bcatenin.ome.tiff'
imc_foxp3 = f'Y:\\coskun-lab\\Efe\\gsr_imc\\datasets\\integration_mass_cytometry_data\\Coregistration\\Coregistration_files\\IMC_images+masks\\Tissue_{tissue}\\channels\\159Tb_FOXP3.ome.tiff'
imc_ki67 = f'Y:\\coskun-lab\\Efe\\gsr_imc\\datasets\\integration_mass_cytometry_data\\Coregistration\\Coregistration_files\\IMC_images+masks\\Tissue_{tissue}\\channels\\152Sm_Ki-67.ome.tiff'
imc_keratin = f'Y:\\coskun-lab\\Efe\\gsr_imc\\datasets\\integration_mass_cytometry_data\\Coregistration\\Coregistration_files\\IMC_images+masks\\Tissue_{tissue}\\channels\\198Pt_Keratin.ome.tiff'
imc_cd45 = f'Y:\\coskun-lab\\Efe\\gsr_imc\\datasets\\integration_mass_cytometry_data\\Coregistration\\Coregistration_files\\IMC_images+masks\\Tissue_{tissue}\\channels\\149Sm_CD45.ome.tiff'

imc_dir_lst = [imc_dna, imc_vimentin, imc_bcat, imc_foxp3, imc_ki67, imc_keratin, imc_cd45]

# Function to adjust image contrast
def enhance_contrast(image):
    # Perform contrast stretching
    p2, p98 = np.percentile(image, (2, 98))
    image_rescale = exposure.rescale_intensity(image, in_range=(p2, p98))
    return image_rescale

imc_img_norm_lst = []
for i in range(len(imc_dir_lst)):
    imc_img = tifffile.imread(imc_dir_lst[i])
    imc_img_enhanced = enhance_contrast(imc_img)
    imc_img_norm = imc_img_enhanced / np.max(imc_img_enhanced)
    imc_img_norm_lst.append(imc_img_norm)
    plt.imshow(imc_img_norm)
    plt.show()

dna_norm = imc_img_norm_lst[0].copy()
vimentin_norm = imc_img_norm_lst[1].copy()
bcat_norm = imc_img_norm_lst[2].copy()
foxp3_norm = imc_img_norm_lst[3].copy()
ki67_norm = imc_img_norm_lst[4].copy()
keratin_norm = imc_img_norm_lst[5].copy()
cd45_norm = imc_img_norm_lst[6].copy()

In [4]:
guide_img = dna_norm.copy()

In [5]:
msi_dir = f'Y:\\coskun-lab\\Efe\\gsr_imc\\datasets\\integration_mass_cytometry_data\\Coregistration\\Coregistration_files\\MALDI_singleTIFF\\Tissue_{tissue}\\'
msi_img_list = []
for msi_img_dir in os.listdir(msi_dir):
    if msi_img_dir == 'Thumbs.db':
        continue
    msi_img = tifffile.imread(msi_dir + msi_img_dir)
    msi_img = np.flip(msi_img)
    msi_img = ((msi_img - msi_img.min()) / (msi_img.max() - msi_img.min()) * 255).astype(np.uint8)
    msi_img_list.append(msi_img)

In [ ]:
for i in range(len(msi_img_list))[:10]:
    print(i)
    plt.imshow(msi_img_list[i])
    plt.axis('off')
    plt.show()

In [ ]:
metabolites_idx = range(len(msi_img_list))
msi_img_list = [msi_img_list[i] for i in metabolites_idx]

for i in range(len(msi_img_list))[:10]:
    print(i)
    plt.imshow(msi_img_list[i])
    plt.show()

In [8]:
def clip_image(data,x_neg_offset,x_pos_offset,y_neg_offset,y_pos_offset):
    return data[y_neg_offset:(data.shape[0]-y_pos_offset),x_neg_offset:(data.shape[1]-x_pos_offset)]

if tissue == 'A_01':
    guide_clip_params = (30, 0, 90, 0)
    msi_clip_params = (0, 3, 0, 12)
elif tissue == 'A_02':
    guide_clip_params = (0, 60, 0, 295)
    msi_clip_params = (9, 0, 53, 0)
elif tissue == 'B_01':
    guide_clip_params = (275, 0, 0, 150)
    msi_clip_params = (0, 52, 25, 0)
elif tissue == 'B_02':
    guide_clip_params = (150, 0, 0, 135)
    msi_clip_params = (0, 27, 21, 0)
elif tissue == 'C_01':
    guide_clip_params = (0, 70, 10, 20)
    msi_clip_params = (1, 0, 0, 0)
elif tissue == 'C_02':
    guide_clip_params = (7, 8, 30, 0)
    msi_clip_params = (0, 0, 0, 1)

guide_img = clip_image(guide_img, *guide_clip_params)

for i in range(len(msi_img_list)):
    msi_img = clip_image(msi_img_list[i], *msi_clip_params)
    msi_img_list[i] = msi_img

In [9]:
metabolites_df = pd.read_excel(f'Y:\\coskun-lab\\Efe\\gsr_imc\\datasets\\integration_mass_cytometry_data\\Coregistration\\Coregistration_files\\MALDI_singleTIFF\\41592_2024_2392_MOESM4_ESM.xlsx')
metabolites_df = metabolites_df[['m/z', 'Short name']]

In [11]:
import numpy as np
from scipy.stats import entropy

def calculate_entropy(image):
    """Calculate the Shannon entropy of a 2D image."""
    pixel_values = image.flatten()
    
    # Get the histogram of pixel values (probability distribution)
    hist, bin_edges = np.histogram(pixel_values, bins=256, density=True)
    
    # Calculate the entropy using the histogram
    return entropy(hist)

def calculate_local_entropy(image, patch_size):
    """Calculate the entropy for patches within the image."""
    h, w = image.shape
    patch_entropies = []
    
    # Divide the image into patches and calculate entropy for each patch
    for i in range(0, h, patch_size):
        for j in range(0, w, patch_size):
            patch = image[i:i+patch_size, j:j+patch_size]
            patch_entropy = calculate_entropy(patch)
            patch_entropies.append(patch_entropy)
    
    # Combine the patch entropies (sum, mean, or max could be used)
    return np.mean(patch_entropies)

def rank_images_by_local_entropy(image_list, patch_size=32):
    """Rank a list of 2D numpy arrays based on their local entropy."""
    entropies = [calculate_local_entropy(img, patch_size) for img in image_list]
    
    # Rank the images based on the entropies (sorting by entropy in descending order)
    sorted_indices = np.argsort(entropies)[::-1]  # Get indices sorted by entropy, descending
    
    # Get the ranked images by using the sorted indices
    ranked_images = [image_list[i] for i in sorted_indices]
    
    return ranked_images, sorted_indices

select_top = False

if select_top == True:
    # Example usage
    patch_size = 16  # Adjust the patch size based on the image resolution
    ranked_msi_img_list, sorted_indices = rank_images_by_local_entropy(msi_img_list, patch_size)
    
    print("Ranked image indices:", sorted_indices)
    
    msi_img_list = ranked_msi_img_list[:40]
    
    plt.figure(figsize=(10,5))
    plt.subplot(1,2,1)
    plt.imshow(guide_img)
    plt.subplot(1,2,2)
    plt.imshow(msi_img_list[0])
    print(guide_img.shape)
    print(msi_img_list[0].shape)    

In [12]:
unique_id = 35
base_dir = f'Y:\\coskun-lab\\Efe and Nishkala\\gsrmsi\\GUIDEDSR\\09_30_2024\\integration_mass_cytometry_data\\{unique_id}'
if not os.path.exists(base_dir):
    os.makedirs(base_dir)

In [13]:
def crop_to_square(img):
    height, width = img.shape[:2]
    
    if height == width:
        return img
    
    if height > width:
        # Crop the height
        start = (height - width) // 2
        end = start + width
        cropped_img = img[start:end, :]
    else:
        # Crop the width
        start = (width - height) // 2
        end = start + height
        cropped_img = img[:, start:end]
    
    return cropped_img

In [ ]:
guide_size = 1024
source_size = 256

guide_resized = cv2.resize(crop_to_square(guide_img), (guide_size, guide_size), interpolation=cv2.INTER_NEAREST)

msi_img_resized_list = []
for i in range(len(msi_img_list)):
    msi_resized = cv2.resize(crop_to_square(msi_img_list[i]), (source_size, source_size), interpolation=cv2.INTER_NEAREST)
    msi_img_resized_list.append(msi_resized)

for i in range(len(msi_img_list))[:10]: 
    plt.imshow(msi_img_resized_list[i])
    plt.show()

In [ ]:
plt.figure(figsize=(10,5))
plt.subplot(1,2,1)
plt.imshow(guide_resized,cmap='gray')
plt.subplot(1,2,2)
plt.imshow(cv2.resize(msi_img_resized_list[0], (guide_resized.shape[1], guide_resized.shape[0]), interpolation=cv2.INTER_NEAREST))

In [ ]:
# Apply Denoising
msi_img_denoised_list = []
for img in msi_img_resized_list:
    msi_img_denoised = cv2.fastNlMeansDenoising(img, None, h=10, templateWindowSize=7, searchWindowSize=21)
    msi_img_denoised_list.append(msi_img_denoised)

# Display denoised images
for i in range(len(msi_img_denoised_list))[:10]:
    plt.imshow(msi_img_denoised_list[i], cmap='viridis')
    plt.axis('off')
    plt.show()

### different spatial regs

In [40]:
import numpy as np
import torch
import gc

iteration_num = 512
epoch = 64
n_fold = 1
custom_params = {
    'model_type': PixTransformNetBase,
    'greyscale': True,
    'channels': -1, 
    'bicubic_input': False,
    'spatial_features_input': True,
    'weights_regularizer': [0, 0.001, 0.001],  # spatial color head
    'loss': 'l1',
    'optim': 'adam',
    'lr': 0.001,
    'batch_size': 32,
    'iteration': iteration_num * epoch,
    'logstep': 512,
    'patch_size': guide_size,
    'stride': 256
}

def PixTransform_with_different_spatial_regs(guide_img, source_img, base_params, spatial_regs, n_fold):
    """
    Apply PixTransform with different spatial regularizer values and return individual predictions and averaged result.
    
    Parameters:
    - guide_img (numpy array): The guiding image for the PixTransform model.
    - source_img (numpy array): The source image to be transformed.
    - base_params (dict): Base parameters for the PixTransform model.
    - spatial_regs (list of float): Different spatial regularizer values to be tested.
    
    Returns:
    - results (dict): A dictionary where the key is the spatial regularizer value, and the value is a list of predictions for each image.
    - averaged_prediction (numpy array): The averaged prediction result from different spatial regularizers.
    """
    predictions = []  # To store predictions for averaging
    results = {str(spatial_reg): [] for spatial_reg in spatial_regs}
    
    for spatial_reg in spatial_regs:
        params = base_params.copy()  # Copy base params to modify
        params['weights_regularizer'][0] = spatial_reg  # Update the spatial regularizer
        
        gc.collect()
        torch.cuda.empty_cache()

        # Perform PixTransform with the current spatial regularizer
        predictions_n_fold = []
        for i in range(n_fold):
            predicted_img_n_fold = PixTransform_diff_models(guide_img=guide_img, source_img=source_img, params=params)
            predictions_n_fold.append(predicted_img_n_fold)
        predicted_img = np.mean(predictions_n_fold, axis=0)
        
        # Append the predicted image to the respective spatial regularizer in the dictionary
        results[str(spatial_reg)].append(predicted_img)
        
        # Store the predicted image for averaging
        predictions.append(predicted_img)
    
    # Compute the average of the predictions across spatial regularizers
    averaged_prediction = np.mean(predictions, axis=0)
    
    return results, averaged_prediction

# Example usage:
denoise = False
enhance = False
spatial_regs = [0.001]  # Different spatial regularizer values to try

'''guide_final = np.zeros((3, guide_size, guide_size))
guide_final[0,:,:] = guide_resized[:,:,0]
guide_final[1,:,:] = guide_resized[:,:,1]
guide_final[2,:,:] = guide_resized[:,:,2]'''
guide_final = guide_resized.copy()
#guide_final = guide_resized_cropped.copy()

# Initialize the dictionary to store individual predictions
individual_predictions_dict = {str(spatial_reg): [] for spatial_reg in spatial_regs}
predicted_target_img_list = []  # List to store the averaged predictions

for i in range(len(msi_img_final_cropped_list)):  # Iterate through the msi images
    gc.collect()
    torch.cuda.empty_cache()
    
    if denoise:
        source_img = msi_img_denoised_list[i]
    if enhance:
        source_img = msi_img_contrast_enhanced_list[i]
    else:
        source_img = msi_img_resized_list[i]
    
    # Get the dictionary of predictions and the averaged prediction for each image
    predictions_dict, averaged_prediction = PixTransform_with_different_spatial_regs(
        guide_final, source_img, custom_params, spatial_regs, n_fold
    )
    
    # Store the individual predictions for each spatial reg
    for spatial_reg in spatial_regs:
        individual_predictions_dict[str(spatial_reg)].append(predictions_dict[str(spatial_reg)][0])
    
    # Store the averaged prediction
    predicted_target_img_list.append(averaged_prediction)

gc.collect()
torch.cuda.empty_cache()

Patch 1/1: 100%|████████████████████████████████████████████████████| 16/16 [02:47<00:00, 10.47s/it, consistency=35.2]


In [ ]:
# Local contrast adjustment functions
import numpy as np
import os
import matplotlib.pyplot as plt
import cv2
import gc

# Set up directories
unique_id = 35
base_dir = f'Y:\\coskun-lab\\Efe and Nishkala\\gsrmsi\\GUIDEDSR\\09_30_2024\\integration_mass_cytometry_data\\{unique_id}'
if not os.path.exists(base_dir):
    os.makedirs(base_dir)

def match_local_statistics_with_details(src, ref, patch_size, stride):
    src_padded = np.pad(src, patch_size // 2, mode='reflect')
    ref_padded = np.pad(ref, patch_size // 2, mode='reflect')
    
    src_matched = np.zeros_like(src)
    
    for i in range(0, src.shape[0], stride):
        for j in range(0, src.shape[1], stride):
            src_patch = src_padded[i:i + patch_size, j:j + patch_size]
            ref_patch = ref_padded[i:i + patch_size, j:j + patch_size]
            
            src_mean, src_std = cv2.meanStdDev(src_patch)
            ref_mean, ref_std = cv2.meanStdDev(ref_patch)
            
            src_patch_normalized = (src_patch - src_mean) / (src_std + 1e-8)
            src_patch_matched = ref_mean + src_patch_normalized * ref_std
            
            src_matched[i:i + stride, j:j + stride] = src_patch_matched[patch_size // 2, patch_size // 2]
    
    return src_matched

def enhance_local_contrast_with_details(src, ref, patch_size=15, stride=1):
    # Step 1: Apply CLAHE
    clahe = cv2.createCLAHE(clipLimit=2.0, tileGridSize=(4, 4))
    src_clahe = clahe.apply((src * 255).astype(np.uint8)) / 255.0
    
    # Step 2: Match local statistics
    src_matched = match_local_statistics_with_details(src_clahe, ref, patch_size, stride)
    
    # Step 3: Compute detail layer
    src_blurred = cv2.GaussianBlur(src, (patch_size, patch_size), 0)
    detail_layer = src - src_blurred
    
    # Step 4: Add detail layer back to the matched image
    result = src_matched + detail_layer
    result = np.clip(result, 0, 1)
    
    return result

# Initialize the dictionary to store combined results and individual predictions
combined_results_adjusted_lst = []
individual_predictions_adjusted_dict = {str(spatial_reg): [] for spatial_reg in spatial_regs}  # Store adjusted individual predictions

for i in range(len(msi_img_resized_list)):  # Iterate through MSI images
    msi_upsampled = cv2.resize(msi_img_resized_list[i], (guide_size, guide_size), interpolation=cv2.INTER_NEAREST)
    msi_upsampled = msi_upsampled.astype(np.float32)
    msi_upsampled /= 255.0
    #msi_upsampled = msi_upsampled.astype(np.float32) / np.iinfo(np.uint16).max

    # Combined predictions across spatial regularizers
    combined = np.zeros((guide_resized.shape[0], guide_resized.shape[1]))
    total_regs = len(spatial_regs)  # Total number of spatial regularizers

    for spatial_reg in spatial_regs:
        individual_pred = individual_predictions_dict[str(spatial_reg)][i].copy()

        # **Save individual spatial reg combinations**
        individual_pred_adjusted = cv2.normalize(individual_pred, None, 0, 1, cv2.NORM_MINMAX).astype('float32')

        # Apply local contrast enhancement with details
        individual_pred_adjusted = enhance_local_contrast_with_details(individual_pred_adjusted, msi_upsampled)

        # Store the adjusted individual prediction
        individual_predictions_adjusted_dict[str(spatial_reg)].append(individual_pred_adjusted)

        # Save the individual prediction with spatial reg details in the filename
        plt.figure()
        plt.imshow(individual_pred_adjusted, cmap="viridis")
        plt.colorbar()
        plt.axis('off')
        plt.savefig(f'{base_dir}\\super_res_msi_{i+1}_{unique_id}_spatial_reg_{str(spatial_reg)}.jpg', dpi=1200, bbox_inches='tight', pad_inches=0)
        np.save(f'{base_dir}\\super_res_msi_{i+1}_{unique_id}_spatial_reg_{str(spatial_reg)}.npy', individual_pred_adjusted)

        # Add individual prediction to the combined result
        combined += individual_pred

    # Normalize the combined output by the total number of spatial regularizers
    combined /= total_regs

    # Apply local contrast enhancement to the combined result
    combined = cv2.normalize(combined, None, 0, 1, cv2.NORM_MINMAX).astype('float32')
    combined_adjusted = enhance_local_contrast_with_details(combined, msi_upsampled)
    combined_adjusted = cv2.normalize(combined_adjusted, None, 0, 1, cv2.NORM_MINMAX).astype('float32')
    combined_results_adjusted_lst.append(combined_adjusted)

    # Save the combined prediction
    plt.figure()
    plt.imshow(combined_adjusted, cmap="viridis")
    plt.colorbar()
    plt.axis('off')
    plt.savefig(f'{base_dir}\\super_res_msi_{i+1}_{unique_id}_combined.jpg', dpi=1200, bbox_inches='tight', pad_inches=0)
    np.save(f'{base_dir}\\super_res_msi_{i+1}_{unique_id}_combined.npy', combined_adjusted)

# Clean up memory after processing
gc.collect()
torch.cuda.empty_cache()

In [246]:
unique_id = 35
base_dir = f'Y:\\coskun-lab\\Efe and Nishkala\\gsrmsi\\GUIDEDSR\\09_30_2024\\integration_mass_cytometry_data\\{unique_id}'
if not os.path.exists(base_dir):
    os.makedirs(base_dir)

params = {'iteration_num*epoch': f'{iteration_num}*{epoch}', 'guide_size': guide_size, 'source_size': source_size}
params_filename = f"{base_dir}\\params_{unique_id}.txt"
with open(params_filename, 'w') as params_file:
    params_file.write(f"unique_id = {unique_id}\n")
    for key, value in params.items():
        params_file.write(f"{key} = {value}\n")

plt.figure() 
plt.imshow(guide_resized, cmap='gray')
plt.axis('off')
plt.savefig(f'{base_dir}\\guide_{unique_id}.jpg', dpi=1200, bbox_inches='tight', pad_inches=0)
plt.close() 

np.save(f'{base_dir}\\guide_{unique_id}.npy', guide_resized)

for i in range(len(msi_img_resized_list)):
    plt.figure()
    msi_upsampled = cv2.resize(msi_img_resized_list[i], (guide_size, guide_size), interpolation=cv2.INTER_NEAREST)
    msi_upsampled = msi_upsampled.astype(np.float32)
    msi_upsampled /= 255.0
    
    plt.imshow(msi_upsampled)
    plt.colorbar()
    plt.axis('off')
    plt.savefig(f'{base_dir}\\low_res_msi_{i+1}_{unique_id}.jpg', dpi=1200, bbox_inches='tight', pad_inches=0)
    plt.close()
    np.save(f'{base_dir}\\low_res_msi_{i+1}_{unique_id}.npy', cv2.resize(msi_img_resized_list[i], (guide_size, guide_size), interpolation=cv2.INTER_NEAREST))

for i in range(len(msi_img_resized_list)):
    plt.figure()
    
    raw_output = cv2.normalize(predicted_target_img_list[i], None, 0, 1, cv2.NORM_MINMAX).astype('float32')
    
    plt.imshow(raw_output)
    plt.colorbar()
    plt.axis('off')
    plt.savefig(f'{base_dir}\\raw_output_{i+1}_{unique_id}.jpg', dpi=1200, bbox_inches='tight', pad_inches=0)
    plt.close()
    np.save(f'{base_dir}\\raw_output_{i+1}_{unique_id}.npy', raw_output)